In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import refspec
from pathlib import Path
import gc
import random

In [ ]:
# Generate x% unique random numbers within a range, inserts random 'glitches'
# Plots & compares all power spectra produced to original power spectrum
# All you need to do is put the file name of your data into file
# Make sure your data file is in the directory you're currently in

file = 'adc_waveform_wPF.bin'
path = os.path.abspath(file)
print(path)
filename=path
data_raw=np.fromfile(filename,dtype=np.int16)

In [ ]:
# Set to 3 hit lengths by default, represented by g
# Set to 4 percentages of number of data points changed to 0 by default, represented by fraction

fraction = [0.01,0.1,0.2,0.3]
glitch_len_max = [1,10,100]
files = [file]
percentages = []
for g in glitch_len_max:
    print ("Doing length:" ,g)
    for f in fraction:
        print ("Doing fraction:",f)
        data = np.copy(data_raw)
        for ndx_s in np.random.randint(0,len(data),int(len(data)*f/(g/2))):
            ndx_e = ndx_s + np.random.randint(0,g+1)
            data[ndx_s:ndx_e]=0
        filename = f"{f}_{g}_{file}"
        files.append(filename)
        open('{}'.format(filename),'wb').write(data.tobytes())
        print("saving '{}'".format(filename))
        if g == glitch_len_max[0]:
            percent = (data==0).sum()/len(data)
            percentages.append(percent)

In [ ]:
#notch is turned off by default in refspec, uncomment the cfg.notch line to turn notch on
print(files)
f = []
pk = []
for x in range(len(files)):
    path = os.path.abspath(files[x])
    print(path)
    filename = path
    cfg = refspec.SpecConfig()
    cfg.Ntaps           = 8
    cfg.Nchannels       = 1
    cfg.Average1Size    = 64
    cfg.Average2Size    = 700
    #cfg.notch = True 
    fundamental         = cfg.fundamental_frequency()
    blocks              = cfg.AverageSize()+2*cfg.Ntaps
    signal = refspec.FileStreamSource(cfg.Nfft,cfg.Nchannels,filename)
    output = refspec.SpecOutput(cfg)
    spectrometer = refspec.RefSpectrometer(signal, cfg)
    spectrometer.run(output)
    f.append(fundamental*np.arange(cfg.Nbins())/1e6)
    pk.append(np.array([output.get_avg_pspec(0, i) for i in range(0, cfg.Nbins())]))

In [ ]:
# indexes: 0 = Original Waveform, 1-4 = hl1, 5-8 = hl10, 9-12 = hl100

from matplotlib.lines import Line2D
true = percentages[0]*100
true2 = percentages[1]*100
true3 = percentages[2]*100
true4 = percentages[3]*100
plt.figure(figsize=(10,5))
plt.plot(f[0], pk[0], 'k', linestyle = "-")
j = 1
plt.plot(f[j],pk[j], 'b', linestyle = "-")
plt.plot(f[j+1],pk[j+1], 'g', linestyle = "-")
plt.plot(f[j+2],pk[j+2], 'y', linestyle = "-")
plt.plot(f[j+3],pk[j+3], 'r', linestyle = "-")
j += 4
plt.plot(f[j],pk[j], 'b', linestyle = "--")
plt.plot(f[j+1],pk[j+1], 'g', linestyle = "--")
plt.plot(f[j+2],pk[j+2], 'y', linestyle = "--")
plt.plot(f[j+3],pk[j+3], 'r', linestyle = "--")
j += 4
plt.plot(f[j],pk[j], 'b', linestyle = "-.")
plt.plot(f[j+1],pk[j+1], 'g', linestyle = "-.")
plt.plot(f[j+2],pk[j+2], 'y', linestyle = "-.")
plt.plot(f[j+3],pk[j+3], 'r', linestyle = "-.")
plt.xlabel('frequency [MHz]')
plt.title('Power Spectrum w/Various Corruptions')
plt.ylabel('power')
plt.loglog()
legend_elements = [Line2D([0], [0], color='k', lw=4, label='No Corruption'),
                   Line2D([0], [0], color='r', lw=4, label='{:.3f}% Corrupt'.format(true4)),
                   Line2D([0], [0], color='y', lw=4, label='{:.3f}% Corrupt'.format(true3)),
                   Line2D([0], [0], color='g', lw=4, label='{:.3f}% Corrupt'.format(true2)),
                   Line2D([0], [0], color='b', lw=4, label='{:.3f}% Corrupt'.format(true)),
                   Line2D([0], [0], color='k', linestyle = "-", lw=1, label='Hit Length 1'),
                   Line2D([0], [0], color='k', linestyle = "--", lw=1, label='Hit Length 10'),
                   Line2D([0], [0], color='k', linestyle = "-.", lw=1, label='Hit Length 100')]
plt.legend(handles=legend_elements)
#plt.savefig('spectrum_comparison.png')
plt.show()